## Imports

In [ ]:
%pip install Bio
from Bio.SeqUtils import IsoelectricPoint
from Bio.Seq import Seq
from Bio.SeqUtils.ProtParam import ProteinAnalysis
import statistics
!apt-get update
!apt-get install emboss
import pandas as pd
import numpy as np
import pandas as pd
import os
from Bio import SeqIO
from typing import Sequence
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
from keras.layers import Conv1D, MaxPooling1D, Flatten, Dropout, Dense, ReLU, Input
from keras.models import Model
from keras.optimizers import Adam
from keras.losses import binary_crossentropy
from keras.activations import sigmoid
import tensorflow as tf
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from keras.losses import categorical_crossentropy
from keras.utils import to_categorical
from sklearn.preprocessing import StandardScaler
from keras.losses import categorical_crossentropy
from keras.utils import to_categorical

## Importing Data

In [70]:
directory = '/content'
data = []

for file in os.listdir(directory):
    if file.endswith('.fa'):
        ix = 0
        for record in SeqIO.parse(os.path.join(directory, file), 'fasta'):
            sequence = str(record.seq)
            function = file.split('_')[0]
            index = f"{function}_{ix}"
            data.append({'Sequence': sequence, 'Class': function, 'Index': index})
            ix += 1

df = pd.DataFrame(data)
df.set_index('Index', inplace=True)


In [71]:
df

,Sequence,Class
Index,,
3_0,MDGEATVKPGEQKVPLWTHGWPPDDPSPSFAAGSSFALPQKRPHPR...,3
3_1,MADLEAVLADVSYLMAMEKSKATPAARASKRIVLPEPSIRSVMQKY...,3
3_2,MVLLRVLILLLSWAAGMGGQYGNPLNKYIRHYEGLSYNVDSLHQKH...,3
3_3,MAGKAAEERGLPKGATPQDTSGLQDRLFSSESDNSLYFTYSGQPNT...,3
3_4,MDVLSPGQGNNTTSPPAPFETGGNTTGISDVTVSYQVITSLLLGTL...,3
...,...,...
8_1195,MSATVVDAVNAAPLSGSKEMSLEEPKKMTREDWRKKKELEEQRKLG...,8
8_1196,MLLADRCRLSTFYEDWSFVMDEERSSMLPTMAAGPNSILFAINIDN...,8
8_1197,MASELAMSNSDLPTSPLAMEYVNDFDLMKFEVKKEPVETDRIISQC...,8


In [72]:
# counter = 0
# for seq in df["Sequence"]:
#   if (len(seq) < 300):
#     counter += 1
# print(counter)

In [76]:
df = df[df['Sequence'].str.len() >= 300]
df = df[df['Class'].isin(["1", "3", "4"])]

df["Class"].value_counts()

3    316
1    301
4    290
Name: Class, dtype: int64

In [77]:
df = df.groupby('Class')

df = df.head(290)
df

,Sequence,Class
Index,,
3_0,MDGEATVKPGEQKVPLWTHGWPPDDPSPSFAAGSSFALPQKRPHPR...,3
3_1,MADLEAVLADVSYLMAMEKSKATPAARASKRIVLPEPSIRSVMQKY...,3
3_2,MVLLRVLILLLSWAAGMGGQYGNPLNKYIRHYEGLSYNVDSLHQKH...,3
3_3,MAGKAAEERGLPKGATPQDTSGLQDRLFSSESDNSLYFTYSGQPNT...,3
3_4,MDVLSPGQGNNTTSPPAPFETGGNTTGISDVTVSYQVITSLLLGTL...,3
...,...,...
1_375,MKLTSEKLPKNPFYASVSQYAAKNQKFFQWKKEKTDYTHANLVDKA...,1
1_377,MATLPAEPSAGPAAGGEAVAAAAATEEEEEEARQLLQTLQAAEGEA...,1
1_378,MAALVLEDGSVLRGQPFGAAVSTAGEVVFQTGMVGYPEALTDPSYK...,1


## Creating the stats table

In [79]:
df = df.reset_index()
df["Sequence"] = df["Sequence"].str.replace("X", "")
df["Sequence"] = df["Sequence"].str.replace("U", "C")

# Iterate through the rows of the DataFrame and create new objects
statistical = []
for i, row in df.iterrows():
    obj_stats = []
    stats = ProteinAnalysis(row["Sequence"])
    count = stats.count_amino_acids()
    percent = stats.get_amino_acids_percent()
    obj_stats.append(round(stats.molecular_weight(),2))
    obj_stats.append(round(stats.aromaticity(),2))
    obj_stats.append(round(stats.instability_index(),2))
    sec_struc = stats.secondary_structure_fraction()  # [helix, turn, sheet]
    obj_stats.append(round(sec_struc[0],2))
    obj_stats.append(round(sec_struc[1],2))
    obj_stats.append(round(sec_struc[2],2))

    obj_stats.append(round(stats.molar_extinction_coefficient()[0],2))
    obj_stats.append(round(stats.charge_at_pH(10),2))
    obj_stats.append(round(stats.charge_at_pH(7),2))
    obj_stats.append(round(stats.charge_at_pH(4),2))

    obj_stats.append(round(stats.isoelectric_point(),2))
    obj_stats.append(round(stats.gravy(),2))

    flex = stats.flexibility()

    obj_stats.append(round(statistics.mean(flex),2))
    obj_stats.append(round(stats.molecular_weight() / sum(count.values()),2))

    obj_stats.append(round(percent["A"] + percent["C"] + percent["G"] \
                         + percent["S"] + percent["T"],2))
    
    obj_stats.append(round(percent["A"] + percent["C"] + percent["D"] \
                         + percent["G"] + percent["N"] + percent["P"] \
                         + percent["S"] + percent["T"] + percent["V"],2))
    
    obj_stats.append(round( percent["A"] + percent["I"] + percent["L"] \
                          + percent["V"] ,2))

    obj_stats.append(round( percent["F"] + percent["H"] + percent["W"] \
                          + percent["Y"] ,2))
    
    obj_stats.append(round( percent["A"] + percent["C"] + percent["F"] \
                          + percent["G"] + percent["I"] + percent["L"] \
                          + percent["M"] + percent["P"] + percent["V"] \
                          + percent["W"] + percent["Y"],2))
    
    
    obj_stats.append(round( percent["D"] + percent["E"] + percent["H"] \
                          + percent["K"] + percent["N"] + percent["Q"] \
                          + percent["R"] + percent["S"] + percent["T"],2))

    obj_stats.append(round( percent["D"] + percent["E"] + percent["H"] \
                          + percent["K"] + percent["R"],2))
    
    obj_stats.append(round( percent["H"] + percent["K"] + percent["R"],2))

    obj_stats.append(round( percent["D"] + percent["E"],2))

    obj_stats.append(round( percent["A"],2))
    obj_stats.append(round( percent["R"],2))
    obj_stats.append(round( percent["N"],2))
    obj_stats.append(round( percent["D"],2))
    obj_stats.append(round( percent["C"],2))
    obj_stats.append(row["Index"])
    obj_stats.append(row["Class"])


    statistical.append(obj_stats)


df2 = pd.DataFrame(statistical, columns=['Weight', 'Aromaticity', 'Instability', \
                                         'Helix', 'Turn', 'Sheet', 'Extinction', \
                                         'Charge10', 'Charge7', 'Charge4', \
                                         'Isoelectric', 'GRAVY', 'Flexibility', \
                                         'AverageWeight', 'Tiny', 'Small', \
                                         'Aliphatic', 'Aromatic', 'NonPolar', \
                                         'Polar', 'Charged', 'Basic', 'Acidic', \
                                         'Ala', 'Arg', 'Asn', 'Asp', 'Cys', \
                                         'Index', "Class"])

df = df.set_index("Index")
df

,index,Sequence,Class
Index,,,
3_0,0,MDGEATVKPGEQKVPLWTHGWPPDDPSPSFAAGSSFALPQKRPHPR...,3
3_1,1,MADLEAVLADVSYLMAMEKSKATPAARASKRIVLPEPSIRSVMQKY...,3
3_2,2,MVLLRVLILLLSWAAGMGGQYGNPLNKYIRHYEGLSYNVDSLHQKH...,3
3_3,3,MAGKAAEERGLPKGATPQDTSGLQDRLFSSESDNSLYFTYSGQPNT...,3
3_4,4,MDVLSPGQGNNTTSPPAPFETGGNTTGISDVTVSYQVITSLLLGTL...,3
...,...,...,...
1_375,865,MKLTSEKLPKNPFYASVSQYAAKNQKFFQWKKEKTDYTHANLVDKA...,1
1_377,866,MATLPAEPSAGPAAGGEAVAAAAATEEEEEEARQLLQTLQAAEGEA...,1
1_378,867,MAALVLEDGSVLRGQPFGAAVSTAGEVVFQTGMVGYPEALTDPSYK...,1


## Implementing types of padding

In [80]:
def pad_sequence(sequence, maxlen, padding='post'):
  num_padding = maxlen - len(sequence)
  padded_sequence = ""

  if padding == 'post':
    padded_sequence = sequence + "0" * (maxlen - len(sequence))

  elif padding == 'extreme':
    half_padding = num_padding // 2
    padded_sequence = "0" * half_padding + sequence + "0" * (num_padding - half_padding)

  elif padding == 'mid':
    sequence = sequence[:600]
    half_sequence = len(sequence) // 2
    padded_sequence = sequence[:half_sequence] + "0" * num_padding + sequence[half_sequence:]
            
  elif padding == 'uniform':
    for i, c in enumerate(sequence[:-1]):
        padded_sequence += c + "0"
        # If there are no more padding characters left, stop interleaving
        if i + 1 == num_padding:
            padded_sequence += sequence[i+1:]
            break
    # If there are still some padding characters left, add them to the end of the string
    padded_sequence += "0" * (num_padding - len(padded_sequence) + len(sequence))

  return padded_sequence

# First stage - perform task 1 and task 2 on the first Dataset with different types of paddings 

In [81]:
df['mid']  = df['Sequence'].apply(lambda x: pad_sequence(x, 600,"mid"))


In [82]:
df

,index,Sequence,Class,mid
Index,,,,
3_0,0,MDGEATVKPGEQKVPLWTHGWPPDDPSPSFAAGSSFALPQKRPHPR...,3,MDGEATVKPGEQKVPLWTHGWPPDDPSPSFAAGSSFALPQKRPHPR...
3_1,1,MADLEAVLADVSYLMAMEKSKATPAARASKRIVLPEPSIRSVMQKY...,3,MADLEAVLADVSYLMAMEKSKATPAARASKRIVLPEPSIRSVMQKY...
3_2,2,MVLLRVLILLLSWAAGMGGQYGNPLNKYIRHYEGLSYNVDSLHQKH...,3,MVLLRVLILLLSWAAGMGGQYGNPLNKYIRHYEGLSYNVDSLHQKH...
3_3,3,MAGKAAEERGLPKGATPQDTSGLQDRLFSSESDNSLYFTYSGQPNT...,3,MAGKAAEERGLPKGATPQDTSGLQDRLFSSESDNSLYFTYSGQPNT...
3_4,4,MDVLSPGQGNNTTSPPAPFETGGNTTGISDVTVSYQVITSLLLGTL...,3,MDVLSPGQGNNTTSPPAPFETGGNTTGISDVTVSYQVITSLLLGTL...
...,...,...,...,...
1_375,865,MKLTSEKLPKNPFYASVSQYAAKNQKFFQWKKEKTDYTHANLVDKA...,1,MKLTSEKLPKNPFYASVSQYAAKNQKFFQWKKEKTDYTHANLVDKA...
1_377,866,MATLPAEPSAGPAAGGEAVAAAAATEEEEEEARQLLQTLQAAEGEA...,1,MATLPAEPSAGPAAGGEAVAAAAATEEEEEEARQLLQTLQAAEGEA...
1_378,867,MAALVLEDGSVLRGQPFGAAVSTAGEVVFQTGMVGYPEALTDPSYK...,1,MAALVLEDGSVLRGQPFGAAVSTAGEVVFQTGMVGYPEALTDPSYK...


## Implementing pI encoding and scaling physico-chemical data

The project takes two approaches for binary and multiclass classification. 

The first approach is to use sequence data and represent it as an array of pIs corresponding to each aminoacid, for example:

QGHEAA = [-1.35, -1.03, 0.59, -3.78, -0.98, -0.98]

In the second approach, ProtParam is run on an input sequence and 28 physico-chemial parameters are extracted

In [83]:
def isoelectric_encoding(sequence):
  alphabet_dict = {'0': 0, 'A': -0.98, 'C': -1.98, 'D': -4.02, 'E': -3.78, \
                   'F': -1.52, 'G': -1.03, 'H': 0.59, 'I': -1.02 ,'K':2.87,\
                   'L': -1.02, 'M': -1.25 , 'N':-1.59 , 'P':-0.70, 'Q':-1.35, \
                   'R': 3.76, 'S':-1.32, 'T':-1.40, 'V':-1.03, 'W':-1.06, 'Y':-1.34}

  vector = np.array([alphabet_dict[char] for char in sequence])
  return vector

In [84]:
df['mid']  = df['mid'].apply(lambda x: isoelectric_encoding(x))


In [85]:
df

,index,Sequence,Class,mid
Index,,,,
3_0,0,MDGEATVKPGEQKVPLWTHGWPPDDPSPSFAAGSSFALPQKRPHPR...,3,"[-1.25, -4.02, -1.03, -3.78, -0.98, -1.4, -1.0..."
3_1,1,MADLEAVLADVSYLMAMEKSKATPAARASKRIVLPEPSIRSVMQKY...,3,"[-1.25, -0.98, -4.02, -1.02, -3.78, -0.98, -1...."
3_2,2,MVLLRVLILLLSWAAGMGGQYGNPLNKYIRHYEGLSYNVDSLHQKH...,3,"[-1.25, -1.03, -1.02, -1.02, 3.76, -1.03, -1.0..."
3_3,3,MAGKAAEERGLPKGATPQDTSGLQDRLFSSESDNSLYFTYSGQPNT...,3,"[-1.25, -0.98, -1.03, 2.87, -0.98, -0.98, -3.7..."
3_4,4,MDVLSPGQGNNTTSPPAPFETGGNTTGISDVTVSYQVITSLLLGTL...,3,"[-1.25, -4.02, -1.03, -1.02, -1.32, -0.7, -1.0..."
...,...,...,...,...
1_375,865,MKLTSEKLPKNPFYASVSQYAAKNQKFFQWKKEKTDYTHANLVDKA...,1,"[-1.25, 2.87, -1.02, -1.4, -1.32, -3.78, 2.87,..."
1_377,866,MATLPAEPSAGPAAGGEAVAAAAATEEEEEEARQLLQTLQAAEGEA...,1,"[-1.25, -0.98, -1.4, -1.02, -0.7, -0.98, -3.78..."
1_378,867,MAALVLEDGSVLRGQPFGAAVSTAGEVVFQTGMVGYPEALTDPSYK...,1,"[-1.25, -0.98, -0.98, -1.02, -1.03, -1.02, -3...."


In [86]:
df2

,Weight,Aromaticity,Instability,Helix,Turn,Sheet,Extinction,Charge10,Charge7,Charge4,...,Charged,Basic,Acidic,Ala,Arg,Asn,Asp,Cys,Index,Class
0,57787.31,0.05,46.83,0.21,0.31,0.26,96940,-15.62,5.37,46.40,...,0.20,0.12,0.09,0.11,0.06,0.02,0.04,0.01,3_0,3
1,79676.74,0.10,46.60,0.30,0.19,0.27,72770,-58.24,2.54,88.65,...,0.32,0.18,0.14,0.05,0.06,0.03,0.06,0.03,3_1,3
2,84141.27,0.09,37.82,0.25,0.26,0.19,74720,-67.65,6.77,89.17,...,0.28,0.16,0.12,0.05,0.06,0.05,0.06,0.05,3_2,3
3,75589.89,0.11,42.20,0.35,0.23,0.27,97750,-33.03,6.37,52.73,...,0.19,0.11,0.08,0.07,0.06,0.03,0.05,0.02,3_3,3
4,46106.36,0.09,36.52,0.34,0.24,0.27,56380,-17.45,12.91,38.73,...,0.19,0.12,0.07,0.10,0.05,0.05,0.04,0.03,3_4,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
865,39779.95,0.12,42.91,0.30,0.17,0.33,60740,-34.02,2.17,40.04,...,0.29,0.16,0.13,0.11,0.05,0.03,0.05,0.02,1_375,1
866,74560.56,0.04,60.19,0.24,0.33,0.36,29910,-41.27,-21.80,33.92,...,0.17,0.07,0.09,0.16,0.04,0.02,0.02,0.01,1_377,1
867,242980.90,0.07,38.03,0.30,0.24,0.28,193440,-144.98,-30.03,187.92,...,0.23,0.12,0.11,0.09,0.06,0.03,0.04,0.02,1_378,1
868,56420.60,0.04,45.28,0.27,0.24,0.29,32430,-38.98,-14.36,36.55,...,0.22,0.11,0.11,0.11,0.04,0.03,0.05,0.02,1_379,1


In [87]:
scaler = StandardScaler()
data_subset = df2.iloc[:,0:28]
scaled_subset = scaler.fit_transform(data_subset)
scaled_df = pd.DataFrame(scaled_subset,columns=data_subset.columns)
scaled_df

,Weight,Aromaticity,Instability,Helix,Turn,Sheet,Extinction,Charge10,Charge7,Charge4,...,NonPolar,Polar,Charged,Basic,Acidic,Ala,Arg,Asn,Asp,Cys
0,-0.479995,-1.007024,-0.159426,-1.310321,1.608531,-0.128542,0.195491,0.730097,0.328115,-0.529650,...,0.278676,-0.278307,-1.023474,-0.640876,-0.846458,1.746474,0.187008,-1.220033,-0.560130,-0.755170
1,-0.176982,0.903379,-0.180337,0.259898,-1.133641,0.116703,-0.152761,0.015385,0.243066,0.105545,...,-0.168096,0.168422,0.974108,1.172334,0.560532,-0.980596,0.187008,-0.513826,0.751615,0.682462
2,-0.115180,0.521298,-0.978589,-0.612446,0.465959,-1.845259,-0.124665,-0.142415,0.370190,0.113362,...,-0.317019,0.317331,0.308248,0.567931,-0.002264,-0.980596,0.187008,0.898587,0.751615,2.120094
3,-0.233556,1.285459,-0.580372,1.132242,-0.219584,0.116703,0.207162,0.438142,0.358168,-0.434484,...,1.023294,-1.022854,-1.189939,-0.943078,-1.127857,-0.071573,0.187008,-0.513826,0.095742,-0.036354
4,-0.641693,0.521298,-1.096782,0.957774,0.008930,0.116703,-0.388916,0.699409,0.554714,-0.644962,...,1.321142,-1.320673,-1.189939,-0.640876,-1.409255,1.291962,-0.313597,0.898587,-0.560130,0.682462
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
865,-0.729269,1.667540,-0.515821,0.259898,-1.590670,1.588174,-0.326095,0.421540,0.231946,-0.625268,...,0.427599,-0.427216,0.474713,0.567931,0.279134,1.746474,-0.313597,-0.513826,0.095742,-0.036354
866,-0.247805,-1.389104,1.055227,-0.786915,2.065559,2.323910,-0.770308,0.299962,-0.488423,-0.717277,...,1.916836,-1.916311,-1.522869,-2.151884,-0.846458,4.019032,-0.814203,-1.220033,-1.871876,-0.755170
867,2.083617,-0.242863,-0.959497,0.259898,0.008930,0.361948,1.585907,-1.439193,-0.735758,1.597989,...,0.874370,-0.873945,-0.524078,-0.640876,-0.283662,0.837451,0.187008,-0.513826,-0.560130,-0.036354
868,-0.498914,-1.389104,-0.300347,-0.263508,0.008930,0.607193,-0.733998,0.338364,-0.264829,-0.677737,...,0.427599,-0.427216,-0.690543,-0.943078,-0.283662,1.746474,-0.814203,-0.513826,0.095742,-0.036354


### The project experiments on three model architectures, with no convolution, with one convolutional layer and with three convolutional layers (implemented from scratch in Keras)

## Architecture #1 - no convolution

In [88]:
def noConv(input_size, hidden_size, num_classes):
    inputs = Input(shape=(input_size,))
    fc1 = Dense(hidden_size, activation='relu')(inputs)
    dropout1 = Dropout(0.5)(fc1)
    fc2 = Dense(hidden_size, activation='relu')(dropout1)
    dropout2 = Dropout(0.25)(fc2)
    fc3 = Dense(num_classes, activation='sigmoid')(dropout2)
    model = Model(inputs, fc3)
    return model



## Architecture #2 - 1 convolutional layer

In [89]:
def oneCNN(input_size, num_classes):

    inputs = Input(shape=(input_size, 1))
    conv_layer1 = Conv1D(32, 3, activation='relu', input_shape=(input_size, 1))(inputs)
    pooling_layer1 = MaxPooling1D(pool_size=2)(conv_layer1)
    dropout1 = Dropout(0.5)(pooling_layer1)
    flatten = Flatten()(conv_layer1)
    fc1 = Dense(16, activation='relu')(flatten)
    fc2 = Dense(8, activation='relu')(fc1)
    fc3 = Dense(num_classes, activation='sigmoid')(fc2)
    model = Model(inputs, fc3)
    return model


## Architecture #3 - stack of 5 convolutional layers

In [90]:
def stackedCNN(input_size, num_classes):
    inputs = Input(shape=(input_size, 1))
    conv_layer1 = Conv1D(32, 2, activation='relu', input_shape=(input_size, 1))(inputs)
    pooling_layer1 = MaxPooling1D(pool_size=2)(conv_layer1)
    conv_layer2 = Conv1D(256, 2, activation='relu')(pooling_layer1)
    dropout1 = Dropout(0.5)(conv_layer2)
    conv_layer3 = Conv1D(128, 2, activation='relu')(dropout1)
    pooling_layer2 = MaxPooling1D(pool_size=2)(conv_layer3)
    conv_layer4 = Conv1D(64, 2, activation='relu')(pooling_layer2)
    dropout2 = Dropout(0.25)(conv_layer4)
    conv_layer5 = Conv1D(32, 2, activation='relu')(dropout2)
    flatten = Flatten()(conv_layer5)
    fc1 = Dense(16, activation='relu')(flatten)
    fc2 = Dense(8, activation='relu')(fc1)
    fc3 = Dense(num_classes, activation='sigmoid')(fc2)
    model = Model(inputs, fc3)
    return model


# Dataset 2 tasks

In [91]:
batch_size = 64
epochs = 45

In [93]:
df = pd.get_dummies(df, columns=['Class'])
df2 = pd.get_dummies(df2, columns=['Class'])


Prepare Data

In [94]:
def split_data(X,y):

  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
  X_train = tf.convert_to_tensor(X_train, dtype=tf.float32)
  y_train = tf.convert_to_tensor(y_train, dtype=tf.float32)
  X_test = tf.convert_to_tensor(X_test, dtype=tf.float32)
  y_test = tf.convert_to_tensor(y_test, dtype=tf.float32)

  return X_train, X_test, y_train, y_test

In [61]:
#task2_sequence
X3 = df["mid"].values
X3 = X3.tolist()
X3 = np.stack(X3)
y3 = df.loc[:,'Class_1':'Class_3'].values


X_train3, X_test3, y_train3, y_test3 = split_data(X3,y3)


#task2_tabular
X4 = scaled_df.values
y4 = df2.loc[:,'Class_1':'Class_3'].values

X_train4, X_test4, y_train4, y_test4 = split_data(X4,y4)



In [96]:
input_tabular = 28
input_sequence = 1000
hidden_size = 64

In [95]:
def run_model(architecture, tabular, out_classes, X_train, X_test, y_train, y_test):

    if architecture == "no_conv":
      if tabular == True:
        model = noConv(input_tabular, hidden_size, out_classes)
      elif tabular == False:
        model = noConv(input_sequence, hidden_size, out_classes)

    if architecture == "one_conv":
      if tabular == True:
        model = oneCNN(input_tabular, out_classes)
      elif tabular == False:
        model = oneCNN(input_sequence, out_classes)

    if architecture == "five_conv":
      if tabular == True:
        model = stackedCNN(input_tabular, out_classes)
      elif tabular == False:
        model = stackedCNN(input_sequence, out_classes)

    if out_classes == 1:
      loss = binary_crossentropy
    else:  
      loss = categorical_crossentropy

    model.compile(optimizer=Adam(), loss = loss, metrics=["accuracy"])
    model.fit(X_train, y_train, batch_size = batch_size, epochs = epochs, verbose=1)
    _, test_acc = model.evaluate(X_test, y_test, verbose=0)
    print(str(out_classes) + " " + str(tabular) + " " + architecture)
    print("Test accuracy:", test_acc)
    model.summary()

    return model

Rewriting this as a loop takes the same amount of space and is less readable:

In [97]:

model = run_model("no_conv", False, 3, X_train3, X_test3, y_train3, y_test3)
model = run_model("no_conv", True,  3, X_train4, X_test4, y_train4, y_test4)


model = run_model("one_conv", False, 3, X_train3, X_test3, y_train3, y_test3)
model = run_model("one_conv", True,  3, X_train4, X_test4, y_train4, y_test4)


model = run_model("five_conv", False, 3, X_train3, X_test3, y_train3, y_test3)
model = run_model("five_conv", True,  3, X_train4, X_test4, y_train4, y_test4)



Epoch 1/45
15/15 [==============================] - 1s 5ms/step - loss: 1.6104 - accuracy: 0.3417
Epoch 2/45
15/15 [==============================] - 0s 5ms/step - loss: 1.1715 - accuracy: 0.4146
Epoch 3/45
15/15 [==============================] - 0s 4ms/step - loss: 1.0998 - accuracy: 0.4062
Epoch 4/45
15/15 [==============================] - 0s 4ms/step - loss: 1.0441 - accuracy: 0.4698
Epoch 5/45
15/15 [==============================] - 0s 4ms/step - loss: 1.0041 - accuracy: 0.5135
Epoch 6/45
15/15 [==============================] - 0s 3ms/step - loss: 0.9552 - accuracy: 0.5208
Epoch 7/45
15/15 [==============================] - 0s 4ms/step - loss: 0.9230 - accuracy: 0.5479
Epoch 8/45
15/15 [==============================] - 0s 4ms/step - loss: 0.8932 - accuracy: 0.5958
Epoch 9/45
15/15 [==============================] - 0s 3ms/step - loss: 0.8404 - accuracy: 0.6104
Epoch 10/45
15/15 [==============================] - 0s 4ms/step - loss: 0.7845 - accuracy: 0.6479
Epoch 11/45
15/15 [